This notebook includes many simulation to study many statisitcal problems

In [97]:
import numpy as np
import pandas as pd
from scipy import stats

In [88]:
rng = np.random.RandomState(8888)

In [92]:
n = 100
true_y = rng.normal(size=n,loc = 0,scale=3)
np.var(true_y)

9.086643338640132

Samples are correlated (some user's measurement were taken multiple times), bootstrap can correct it (assuming `true_y` is I.I.D)

__Recall from A/B Testing books: Having the randomization unit be coarser than the analysis uniut, such as randomizing by user and the click-through rate (by page) , will work, but requires more nuanced anlayses methods such as bootstrap or the delta method__.

In [95]:
n_replicates = 1000
out = rng.choice(true_y,size=(n_replicates,n))

In [94]:
np.mean(out.var(axis=1))

9.040614450723973

__Outlier effect on A/B testing, p-value of 0.32 and T-stat of 1.__

https://statisticaloddsandends.wordpress.com/2020/10/01/p-value-around-0-32-you-may-have-a-problem-with-outliers/

In [151]:
n = 100
a = rng.uniform(size = n)
b = rng.uniform(size = n)
stats.ttest_1samp(a,popmean= 1/2 )

Ttest_1sampResult(statistic=-0.41372975536954076, pvalue=0.6799669338684812)

#### still not significant due to cancellation of the outlier in both numerator and denominator

In [155]:
a[-1] = 10000
stats.ttest_1samp(a,popmean=1/2)

Ttest_1sampResult(statistic=0.999871774925432, pvalue=0.319810219439993)

In [154]:
stats.ttest_1samp(a-b,popmean=0)

Ttest_1sampResult(statistic=1.000424968301513, pvalue=0.3195438921705351)

__T-stats__

In [128]:
a.mean()/(a.std()/np.sqrt(n))

1.0101943786644874